In [ ]:
from tqdm import tqdm
import numpy as np
from copy import deepcopy
from math import gcd

## Part 1

In [ ]:
class Signal:
    def __init__(self, receiving_node,sender, freq):
        self.rec_node = receiving_node
        self.sender = sender
        self.freq = freq
        
    def __repr__(self):
        return 'from ' + str(self.sender) + ' towards ' + self.rec_node + ', ' + self.freq

In [ ]:
# % is a flip-flop
# & is a conjunction
class Node:
    def __init__(self, node_type, name, connections):
        self.node_type = node_type
        self.name = name
        self.connections = connections
    
    def __repr__(self):
        return '|' + self.node_type + ', ' + self.name + ', ' + str(self.connections) + '|'
    
    def transmit_pulse(self, signal):
        output_signals_list = []
        for connection in self.connections:
            output_signals_list.append(Signal(connection, self.name, 'low'))
        return output_signals_list

In [ ]:
class broadcast(Node):
    def __init__(self, node_type, name, connections):
        self.node_type = node_type
        self.name = name
        self.connections = connections
       
    def __repr__(self):
        return '|' + self.node_type + ', ' + self.name + ', ' + str(self.connections) + '|'
    
    def transmit_pulse(self, signal):
        output_signals_list = []
        for connection in self.connections:
            output_signals_list.append(Signal(connection, self.name, 'low'))
        return output_signals_list

In [ ]:
# Flip-flop modules (prefix %) are either on or off; 
# they are initially off. 
# If a flip-flop module receives a high pulse, it is ignored and nothing happens. 
# However, if a flip-flop module receives a low pulse, it flips between on and off. 
# If it was off, it turns on and sends a high pulse. 
# If it was on, it turns off and sends a low pulse.

class flip_flop(Node):
    def __init__(self, node_type, name, connections):
        self.node_type = node_type
        self.name = name
        self.connections = connections
        self.is_on = False

    def transmit_pulse(self, signal) -> []:
        if signal.freq == 'high':
            return []
        # first sends out signals
        output_freq = 'low' if self.is_on else 'high' 
        output_signals_list = []
        for connection in self.connections:
            output_signals_list.append(Signal(connection, self.name, output_freq))
        # then updates its status
        self.is_on = not self.is_on
        return output_signals_list

    def __repr__(self):
        return '|' + self.node_type + ', ' + self.name + ', ' + str(self.connections) + ', is_on = ' + str(self.is_on) + '|'    

In [ ]:
# Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; 
# they initially default to remembering a low pulse for each input.
# When a pulse is received, the conjunction module first updates its memory for that input. 
# Then, if it remembers high pulses for all inputs, it sends a low pulse; 
# otherwise, it sends a high pulse.

class conjunction(Node):
    def __init__(self, node_type, name, connections):
        self.node_type = node_type
        self.name = name
        self.connections = connections
        self.origins = {}
        
    def add_origins(self, origin_list):
        for origin in origin_list:
            self.origins[origin] = 'low'
       
    def transmit_pulse(self, signal):
        
        #updates last signal received from sender
        self.origins[signal.sender] = signal.freq
        
        # check what sort of output
        all_high = all( freq == 'high' for freq in list(self.origins.values()) ) 
                
        # sends out signals
        output_freq = 'low' if all_high else 'high' 
        output_signals_list = []
        for connection in self.connections:
            output_signals_list.append(Signal(connection, self.name, output_freq))
        # then updates its status
        return output_signals_list
    
    def __repr__(self):
        return '|' + self.node_type + ', ' + self.name + ', ' + str(self.connections) + ', origins = '+ str(self.origins) + '|'

In [ ]:
nodes = {}
nodes['rx']= Node('exit', 'rx', [])
nodes['ou']= Node('exit', 'ou', [])

# generate the nodes with output connections
with open('Day20_input.txt') as f:
    for line in tqdm(f):
        if line[0] != 'b':
            node_type = line[0]
            name = line[1:3]
            connections = line[7:].strip('\n').replace(',', '').split(' ')
            if node_type == '%':
                new_node = flip_flop(node_type, name, connections)
            else:
                new_node = conjunction(node_type, name, connections)
        else: 
            node_type = 'b'
            name = 'broadcaster'
            connections = line[15:].strip('\n').replace(',', '').split(' ')
            new_node = broadcast(node_type, name, connections)
        nodes[name] = new_node
        #print(new_node)

# add origin nodes to conjunction nodes
for node in nodes.values():
    if node.node_type == '&':
        for orig_node in nodes.values():
            if node.name in orig_node.connections:
                node.add_origins([orig_node.name])

                
n_low_signals  = 0
n_high_signals = 0
               
for it in tqdm(range(1000)):
    signal_queue = [Signal('broadcaster',None, 'low')]
    while signal_queue and signal_queue[0]:
        new_signal = signal_queue.pop(0)
        if new_signal.freq == 'high':
            n_high_signals +=1
        else:
            n_low_signals +=1
        signal_queue += nodes[new_signal.rec_node].transmit_pulse(new_signal) 


print('total low signals = ', n_low_signals)
print('total high signals = ', n_high_signals)
print(n_low_signals*n_high_signals)

## Part 2

In [ ]:
nodes = {}
nodes['rx']= Node('exit', 'rx', [])
nodes['ou']= Node('exit', 'ou', [])

# generate the nodes with output connections
with open('Day20_input.txt') as f:
    for line in tqdm(f):
        if line[0] != 'b':
            node_type = line[0]
            name = line[1:3]
            connections = line[7:].strip('\n').replace(',', '').split(' ')
            if node_type == '%':
                new_node = flip_flop(node_type, name, connections)
            else:
                new_node = conjunction(node_type, name, connections)
        else: 
            node_type = 'b'
            name = 'broadcaster'
            connections = line[15:].strip('\n').replace(',', '').split(' ')
            new_node = broadcast(node_type, name, connections)
        nodes[name] = new_node

# add origin nodes to conjunction nodes
for node in nodes.values():
    if node.node_type == '&':
        for orig_node in nodes.values():
            if node.name in orig_node.connections:
                node.add_origins([orig_node.name])



rx_received_low = False
n_button_pressed = 0
rv_sends_high = np.array([])
vp_sends_high = np.array([])
dc_sends_high = np.array([])
cq_sends_high = np.array([])



# we check in the first 10000
for n_button_pressed in range(1,20000):
    signal_queue = [Signal('broadcaster',None, 'low')]
    while signal_queue and signal_queue[0]:
        new_signal = signal_queue.pop(0)
        
        # this is not going to happen
        if new_signal.rec_node == 'rx' and new_signal.freq == 'low':
            rx_received_low = True
            break
        
        # these four nodes are connected to 'ns', which in turn is the one connected to 'rx'
        # let us check when they are firing a 'high'...
        if new_signal.sender == 'rv' and new_signal.freq == 'high':
            rv_sends_high = np.append(rv_sends_high, int(n_button_pressed))
            
        if new_signal.sender == 'vp' and new_signal.freq == 'high':
            vp_sends_high = np.append(vp_sends_high, int(n_button_pressed))
            
        if new_signal.sender == 'dc' and new_signal.freq == 'high':
            dc_sends_high = np.append(dc_sends_high, int(n_button_pressed))
            
        if new_signal.sender == 'cq' and new_signal.freq == 'high':
            cq_sends_high = np.append(cq_sends_high, int(n_button_pressed))
            
        signal_queue += nodes[new_signal.rec_node].transmit_pulse(new_signal) 
    
    
# check if they have a regular occurrence
rv_diff = rv_sends_high[1:] - rv_sends_high[:-1]
vp_diff = vp_sends_high[1:] - vp_sends_high[:-1]
dc_diff = dc_sends_high[1:] - dc_sends_high[:-1]
cq_diff = cq_sends_high[1:] - cq_sends_high[:-1]

# would you look at that
print(rv_diff)
print(vp_diff)
print(dc_diff)
print(cq_diff)

# compute lcm of these periodic events. Since the system started in a reset position, 
# and the AoC people are not sadistic, one hopes that the period started at n_buttons_pressed = 0
periods = [int(rv_diff[0]), int(vp_diff[0]), int(dc_diff[0]), int(cq_diff[0])]
lcm = 1
for i in periods:
    lcm = lcm*i//gcd(lcm, i)
print('first time all 4 nodes fire high to NS is at iteration ',lcm)